In [20]:
import glob, os, re
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split

In [21]:
txt_path = 'indi_text.txt'
raw_corpus = []

with open(txt_path, 'r', encoding = 'utf-8') as f:
    raw = f.read().splitlines()
    raw_corpus.extend(raw)
print(len(raw_corpus))

433820


In [22]:
raw_corpus[:5]

['돌아서는 너를 보며', '', '난 아무 말도 할 수 없었고', '', '슬퍼하기엔 짧았던']

In [23]:
print(len(raw_corpus[1]))

0


In [24]:
print(len(max(raw_corpus, key = len)))

186


In [25]:
#raw_corpus = ' '.join(raw_corpus).split()
#raw_corpus = list(filter(None, raw_corpus))
print(len(raw_corpus))
print(raw_corpus[:5])

433820
['돌아서는 너를 보며', '', '난 아무 말도 할 수 없었고', '', '슬퍼하기엔 짧았던']


In [26]:
print(raw_corpus[-5:])

['잠이 안 와 미쳤나봐', '', '복잡하고 답답하니까', '', '그냥 널 생각하고 있어']


In [27]:
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z가-힣?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = re.sub(r"\(.\)", " ", sentence) # 7
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [28]:
print(preprocess_sentence("This @_is ;;;sample        (sentences) sentence . 그리고 "))

<start> This is sample sentences sentence . 그리고 <end>


In [29]:
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if sentence[-1] == ':':
        continue
    if len(sentence)>150:
        continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
corpus

['<start> 돌아서는 너를 보며 <end>',
 '<start> 난 아무 말도 할 수 없었고 <end>',
 '<start> 슬퍼하기엔 짧았던 <end>',
 '<start> 나의 해는 저물어 갔네 <end>',
 '<start> 지나치는 모진 기억이 <end>',
 '<start> 바람 따라 흩어질 때면 <end>',
 '<start> 아무 일도 없듯이 보내주려 해 <end>',
 '<start> 아픈 맘이 남지 않도록 <end>',
 '<start> 안녕 멀어지는 나의 하루야 <end>',
 '<start> 빛나지 못한 나의 별들아 <end>',
 '<start> 차마 아껴왔던 말 이제서야 <end>',
 '<start> 잘 지내 인사를 보낼 게 <end>',
 '<start> 떠나가는 너를 보며 <end>',
 '<start> 난 아무 말도 할 수 없었고 <end>',
 '<start> 슬퍼하기엔 짧았던 <end>',
 '<start> 나의 해는 저물어 갔네 <end>',
 '<start> 돌이킬 순 없는 추억이 <end>',
 '<start> 바람 따라 흩어질 때면 <end>',
 '<start> 아무 일도 없듯이 보내주려 해 <end>',
 '<start> 아픈 맘이 남지 않도록 <end>',
 '<start> 안녕 멀어지는 나의 하루야 <end>',
 '<start> 빛나지 못한 나의 별들아 <end>',
 '<start> 차마 아껴왔던 말 이제서야 <end>',
 '<start> 잘 지내 인사를 보낼 게 <end>',
 '<start> 잘 지내 인사를 보낼 게 <end>',
 '<start> 나는 읽기 쉬운 마음이야 <end>',
 '<start> 당신도 스윽 훑고 가셔요 <end>',
 '<start> 달랠 길 없는 외로운 마음 있지 <end>',
 '<start> 머물다 가셔요 음 <end>',
 '<start> 내게 긴 여운을 남겨줘요 <end>',
 '<start> 사랑을 사랑을 해줘요 <end>',
 '<start> 할 수 있다면 그

In [19]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 12000, filters = ' ', oov_token = '<unk>')
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post')
    print('토크나이저: ', tokenizer, '\n', tensor)
    
    return tensor, tokenizer

In [12]:
tensor, tokenizer = tokenize(corpus)

토크나이저:  <keras.src.legacy.preprocessing.text.Tokenizer object at 0x0000022764627C50> 
 [[    2  4361    22 ...     0     0     0]
 [    2     7   164 ...     0     0     0]
 [    2     1  5444 ...     0     0     0]
 ...
 [    2   472    74 ...     0     0     0]
 [    2 10161     1 ...     0     0     0]
 [    2    80     8 ...     0     0     0]]


In [13]:
tensor.shape

(210237, 27)

In [14]:
tensor[0]

array([   2, 4361,   22,  334,    3,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])

In [15]:
for idx in tokenizer.index_word:
    print(idx, ':', tokenizer.index_word[idx])
    if idx>=20: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : you
6 : 내
7 : 난
8 : 널
9 : 내가
10 : 날
11 : 수
12 : 그
13 : me
14 : the
15 : 더
16 : 이
17 : 너의
18 : 다
19 : 나는
20 : my


In [16]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]
print('텐서 길이: ', tensor.shape)
print('소스문장 길이: ', len(src_input[0]))
print('타겟문장 길이: ', len(tgt_input[0]))

텐서 길이:  (210237, 27)
소스문장 길이:  26
타겟문장 길이:  26


In [17]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 1234)
print('Source Train 길이: ', enc_train.shape)
print('Target Train 길이: ', dec_train.shape)
print('Source Test 길이: ', enc_val.shape)
print('Target Test 길이: ', dec_val.shape)

Source Train 길이:  (168189, 26)
Target Train 길이:  (168189, 26)
Source Test 길이:  (42048, 26)
Target Test 길이:  (42048, 26)


In [18]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epochs = len(src_input) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1

dataset_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset_train = dataset_train.shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder = True)

dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_val = dataset_val.shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder = True)

print(dataset_train)
print(dataset_val)

<_BatchDataset element_spec=(TensorSpec(shape=(256, 26), dtype=tf.int32, name=None), TensorSpec(shape=(256, 26), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(256, 26), dtype=tf.int32, name=None), TensorSpec(shape=(256, 26), dtype=tf.int32, name=None))>


In [19]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [20]:
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)
model

<TextGenerator name=text_generator, built=False>

In [21]:
for src_sample, tgt_sample in dataset_train.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 26, 12001), dtype=float32, numpy=
array([[[-1.66465630e-04, -1.23079459e-04, -2.28258883e-04, ...,
         -2.78883090e-04,  2.89886928e-04, -2.11112929e-04],
        [-2.85402202e-04, -2.06569792e-04, -3.19867453e-04, ...,
         -3.73350194e-04,  2.56283092e-04, -4.33517882e-04],
        [-6.09326584e-04, -1.68034516e-04,  5.89593255e-05, ...,
         -4.34003829e-04,  1.72316068e-04, -6.30301307e-04],
        ...,
        [ 2.68865842e-05,  1.69434398e-03, -5.44381677e-04, ...,
          8.87255999e-04, -3.14755645e-03, -2.68209632e-03],
        [ 7.81719573e-05,  1.71291549e-03, -5.74341044e-04, ...,
          8.93402379e-04, -3.18085612e-03, -2.79565714e-03],
        [ 1.26064289e-04,  1.72861898e-03, -5.96068450e-04, ...,
          8.94417637e-04, -3.20531265e-03, -2.89201364e-03]],

       [[-1.66465630e-04, -1.23079459e-04, -2.28258883e-04, ...,
         -2.78883090e-04,  2.89886928e-04, -2.11112929e-04],
        [-2.19884212e-04, -1.03789280e-04, -2

In [22]:
model.summary()

Model: "text_generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     3,072,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │     8,392,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │    12,301,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,012,961 (110.68 MB)

 Trainable params: 29,012,961 (110.68 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')

In [24]:
model.compile(loss = loss, optimizer = optimizer)
model.fit(dataset_train, epochs = 10)

Epoch 1/10


  1/656 ━━━━━━━━━━━━━━━━━━━━ 26:23:40 145s/step - loss: 9.3930